# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [23]:
# import libraries
import nltk
import pandas as pd
from sqlalchemy import create_engine
import re
import pickle
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('Messages', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df["message"]
Y = df.drop(["id", "message", "original", "genre"], axis=1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9]+", " ", text)
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if not w in stopwords.words('english')]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=4), n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [7]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001A2444DD5E0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=4),
                                       n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
Y_pred = pipeline.predict(X_test)

In [9]:
print(classification_report(Y_test, Y_pred, target_names=Y.columns.values))
print("Total accuracy:")
print((Y_pred == Y_test).mean().mean())

                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      4012
               request       0.82      0.49      0.61       943
                 offer       0.00      0.00      0.00        29
           aid_related       0.76      0.69      0.72      2172
          medical_help       0.59      0.08      0.14       448
      medical_products       0.84      0.16      0.26       277
     search_and_rescue       0.89      0.05      0.09       168
              security       0.50      0.01      0.02        94
              military       0.77      0.14      0.24       160
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.36      0.52       352
                  food       0.81      0.67      0.73       537
               shelter       0.83      0.35      0.49       469
              clothing       0.82      0.12      0.21        76
                 money       0.83      

d:\git\udacity-disaster-response-pipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\git\udacity-disaster-response-pipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\git\udacity-disaster-response-pipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
    "clf__estimator__n_estimators": [50, 100, 200],
    "clf__estimator__min_samples_split": [2, 4]
}

In [11]:
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, cv=3)

In [12]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV 1/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50;, score=0.252 total time= 1.5min
[CV 2/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50;, score=0.252 total time= 1.5min
[CV 3/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50;, score=0.265 total time= 1.5min
[CV 1/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100;, score=0.255 total time= 1.9min
[CV 2/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100;, score=0.255 total time= 2.0min
[CV 3/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100;, score=0.269 total time= 2.0min
[CV 1/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200;, score=0.256 total time= 2.7min
[CV 2/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200;, score=0.257 total time= 2.7min
[CV 3/3] END cl

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001A2444DD5E0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=4),
                                                              n_jobs=-1))]),
             param_grid={'clf__estimator__min_samples_split': [2, 4],
                         'clf__estimator__n_estimators': [50, 100, 200]},
             verbose=3)

In [13]:
cv.best_params_

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 200}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
Y_pred = cv.predict(X_test)

In [15]:
print(classification_report(Y_test, Y_pred, target_names=Y.columns.values))
print("Total accuracy:")
print((Y_pred == Y_test).mean().mean())

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4012
               request       0.83      0.48      0.61       943
                 offer       0.00      0.00      0.00        29
           aid_related       0.76      0.70      0.73      2172
          medical_help       0.56      0.07      0.13       448
      medical_products       0.87      0.12      0.22       277
     search_and_rescue       0.86      0.04      0.07       168
              security       0.00      0.00      0.00        94
              military       0.88      0.09      0.17       160
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.37      0.52       352
                  food       0.83      0.64      0.73       537
               shelter       0.81      0.37      0.51       469
              clothing       0.83      0.13      0.23        76
                 money       0.83      

d:\git\udacity-disaster-response-pipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\git\udacity-disaster-response-pipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\git\udacity-disaster-response-pipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
pipeline_2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(MultinomialNB(), n_jobs=-1))
])

In [17]:
parameters_2 = {
    "clf__estimator__fit_prior": [False, True],
    "clf__estimator__alpha": [0.1, 0.5, 1.0, 2.0], 
}

In [18]:
cv_2 = GridSearchCV(pipeline_2, param_grid=parameters_2, verbose=3, cv=3)

In [19]:
cv_2.fit(X_train, Y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3] END clf__estimator__alpha=0.1, clf__estimator__fit_prior=False;, score=0.118 total time= 1.2min
[CV 2/3] END clf__estimator__alpha=0.1, clf__estimator__fit_prior=False;, score=0.123 total time= 1.2min
[CV 3/3] END clf__estimator__alpha=0.1, clf__estimator__fit_prior=False;, score=0.127 total time= 1.2min
[CV 1/3] END clf__estimator__alpha=0.1, clf__estimator__fit_prior=True;, score=0.207 total time= 1.2min
[CV 2/3] END clf__estimator__alpha=0.1, clf__estimator__fit_prior=True;, score=0.204 total time= 1.2min
[CV 3/3] END clf__estimator__alpha=0.1, clf__estimator__fit_prior=True;, score=0.216 total time= 1.2min
[CV 1/3] END clf__estimator__alpha=0.5, clf__estimator__fit_prior=False;, score=0.191 total time= 1.2min
[CV 2/3] END clf__estimator__alpha=0.5, clf__estimator__fit_prior=False;, score=0.189 total time= 1.2min
[CV 3/3] END clf__estimator__alpha=0.5, clf__estimator__fit_prior=False;, score=0.203 total time= 1.2mi

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001A2444DD5E0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=MultinomialNB(),
                                                              n_jobs=-1))]),
             param_grid={'clf__estimator__alpha': [0.1, 0.5, 1.0, 2.0],
                         'clf__estimator__fit_prior': [False, True]},
             verbose=3)

In [20]:
cv_2.best_params_

{'clf__estimator__alpha': 0.1, 'clf__estimator__fit_prior': True}

In [21]:
Y_pred = cv_2.predict(X_test)

In [22]:
print(classification_report(Y_test, Y_pred, target_names=Y.columns.values))
print("Total accuracy:")
print((Y_pred == Y_test).mean().mean())

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4012
               request       0.72      0.55      0.62       943
                 offer       0.00      0.00      0.00        29
           aid_related       0.69      0.68      0.68      2172
          medical_help       0.49      0.12      0.20       448
      medical_products       0.71      0.15      0.25       277
     search_and_rescue       0.00      0.00      0.00       168
              security       0.25      0.01      0.02        94
              military       0.68      0.24      0.36       160
           child_alone       0.00      0.00      0.00         0
                 water       0.53      0.09      0.16       352
                  food       0.67      0.32      0.43       537
               shelter       0.65      0.17      0.27       469
              clothing       0.62      0.13      0.22        76
                 money       0.60      

d:\git\udacity-disaster-response-pipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\git\udacity-disaster-response-pipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\git\udacity-disaster-response-pipeline\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

The Naive Bayes classifier performs worse than the Random Forest Classifier. I will stick with Random Forest therefore.

### 9. Export your model as a pickle file

In [24]:
with open('classifier_model.pkl', 'wb') as file:
    pickle.dump(cv.best_estimator_, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.